In [2]:
from utils import *
import numpy as np
import pickle

In [3]:
t,v = get_data()

In [4]:
ui_mtx = create_ui_matrix(t)

In [5]:
ui_mtx

array([[4, 5, 5, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [44]:
rel_cols = ui_mtx[:,[6,1]]
rel_cols = rel_cols[(rel_cols!=0).all(axis=1)]
rel_cols

array([[4., 5.],
       [5., 3.],
       [4., 3.]])

In [45]:
c = rel_cols[:,0] - rel_cols[:,1]
c.mean()

0.6666666666666666

In [ ]:
t[t[:,2] == 0]

array([], shape=(0, 3), dtype=int32)

In [47]:
test_dic = {'a':1,'b':2,'c':3}
with open('dic_test.pickle', 'wb') as f:
    pickle.dump(test_dic, f)

In [48]:
with open('dic_test.pickle', 'rb') as f:
    moti = pickle.load(f)

In [49]:
moti

{'a': 1, 'b': 2, 'c': 3}